# ChatGPT 모델에 입력 형식을 지정하는 방법

ChatGPT는 OpenAI의 최신 모델인 `gpt-3.5-turbo`와 `gpt-4`로 구동됩니다.

OpenAI API를 사용하여 `gpt-3.5-turbo` 또는 `gpt-4`로 자신만의 애플리케이션을 구축할 수 있습니다.

채팅 모델은 일련의 메시지를 입력으로 받아 AI가 작성한 메시지를 출력으로 반환합니다.

이 가이드에서는 몇 가지 API 호출 예제를 통해 채팅 형식을 설명합니다.

## 1. openai 라이브러리 가져오기

In [ ]:
# if needed, install and/or upgrade to the latest version of the OpenAI Python library
%pip install --upgrade openai


In [2]:
# import the OpenAI Python library for calling the OpenAI API
import openai


## 2. 채팅 API 호출 예시

채팅 API 호출에는 두 가지 필수 입력이 있습니다:
- model`: 사용하려는 모델의 이름(예: `gpt-3.5-turbo`, `gpt-4`, `gpt-3.5-turbo-0613`, `gpt-3.5-turbo-16k-0613`).
- 메시지`: 메시지 객체의 목록으로, 각 객체에는 두 개의 필수 필드가 있습니다:
    - role`: 메신저의 역할(`시스템`, `사용자`, `어시스턴트` 중 하나)
    - content`: 메시지의 내용(예: `아름다운 시를 써줘`)

메시지에는 선택적 `name` 필드가 포함되어 메신저의 이름을 지정할 수도 있습니다. 예: `예시-사용자`, `알리스`, `블랙베어봇`. 이름에는 공백을 포함할 수 없습니다.

2023년 6월부터, 선택적으로 함수에 공급할 JSON을 생성할 수 있는지 여부를 GPT에 알려주는 `함수` 목록을 제출할 수도 있습니다. 자세한 내용은 [문서](https://platform.openai.com/docs/guides/gpt/function-calling), [API 참조](https://platform.openai.com/docs/api-reference/chat) 또는 쿡북 가이드 [채팅 모델로 함수를 호출하는 방법](How_to_call_functions_with_chat_models.ipynb)을 참조하세요.

일반적으로 대화는 어시스턴트에게 행동 방법을 알려주는 시스템 메시지로 시작한 다음 사용자와 어시스턴트 메시지가 번갈아 가며 나오지만, 반드시 이 형식을 따라야 할 필요는 없습니다.

채팅 API 호출 예시를 통해 채팅 형식이 실제로 어떻게 작동하는지 살펴보기로 하죠.

In [3]:
# Example OpenAI Python library request
MODEL = "gpt-3.5-turbo"
response = openai.ChatCompletion.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Knock knock."},
        {"role": "assistant", "content": "Who's there?"},
        {"role": "user", "content": "Orange."},
    ],
    temperature=0,
)

response


<OpenAIObject chat.completion id=chatcmpl-7UkgnSDzlevZxiy0YjZcLYdUMz5yZ at 0x118e394f0> JSON: {
  "id": "chatcmpl-7UkgnSDzlevZxiy0YjZcLYdUMz5yZ",
  "object": "chat.completion",
  "created": 1687563669,
  "model": "gpt-3.5-turbo-0301",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Orange who?"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 39,
    "completion_tokens": 3,
    "total_tokens": 42
  }
}

보시다시피 응답 객체에는 몇 가지 필드가 있습니다:
- id`: 요청의 ID
- object`: 반환된 객체의 유형(예: `chat.completion`)
- created`: 요청의 타임스탬프
- model`: 응답을 생성하는 데 사용된 모델의 전체 이름
- 사용량`: 응답을 생성하는 데 사용된 토큰 수, 프롬프트, 완료 및 총계를 계산합니다.
- choices`: 완료 객체 목록(`n`을 1보다 크게 설정하지 않는 한 하나만)
    - 메시지`: 모델에 의해 생성된 메시지 객체로, `역할` 및 `내용`이 포함됩니다.
    - finish_reason`: 모델이 텍스트 생성을 중지한 이유(`stop`, 또는 `max_tokens` 제한에 도달한 경우 `length`)
    - index`: 선택 목록에서 완료된 항목의 색인

를 사용하여 회신만 추출합니다:

In [4]:
response['choices'][0]['message']['content']


'Orange who?'

대화 기반이 아닌 작업도 첫 번째 사용자 메시지에 지침을 배치하여 채팅 형식에 맞출 수 있습니다.

예를 들어 모델에게 해적 검은수염의 스타일로 비동기 프로그래밍을 설명해 달라고 요청하려면 다음과 같이 대화를 구성할 수 있습니다:

In [5]:
# example with a system message
response = openai.ChatCompletion.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Explain asynchronous programming in the style of the pirate Blackbeard."},
    ],
    temperature=0,
)

print(response['choices'][0]['message']['content'])


Ahoy matey! Asynchronous programming be like havin' a crew o' pirates workin' on different tasks at the same time. Ye see, instead o' waitin' for one task to be completed before startin' the next, ye can assign tasks to yer crew and let 'em work on 'em simultaneously. This way, ye can get more done in less time and keep yer ship sailin' smoothly. It be like havin' a bunch o' pirates rowin' the ship at different speeds, but still gettin' us to our destination. Arrr!


In [6]:
# example without a system message
response = openai.ChatCompletion.create(
    model=MODEL,
    messages=[
        {"role": "user", "content": "Explain asynchronous programming in the style of the pirate Blackbeard."},
    ],
    temperature=0,
)

print(response['choices'][0]['message']['content'])


Ahoy mateys! Let me tell ye about asynchronous programming, arrr! It be like havin' a crew of sailors workin' on different tasks at the same time, without waitin' for each other to finish. Ye see, in traditional programming, ye have to wait for one task to be completed before movin' on to the next. But with asynchronous programming, ye can start multiple tasks at once and let them run in the background while ye focus on other things.

It be like havin' a lookout keepin' watch for enemy ships while the rest of the crew be busy with their own tasks. They don't have to stop what they're doin' to keep an eye out, because the lookout be doin' it for them. And when the lookout spots an enemy ship, they can alert the crew and everyone can work together to defend the ship.

In the same way, asynchronous programming allows different parts of yer code to work together without gettin' in each other's way. It be especially useful for tasks that take a long time to complete, like loadin' large file

## 3. gpt-3.5-turbo-0301 지침을 위한 팁

모델 지시를 위한 모범 사례는 모델 버전에 따라 변경될 수 있습니다. 다음 조언은 'gpt-3.5-turbo-0301'에 적용되며 향후 모델에는 적용되지 않을 수 있습니다.

시스템 메시지 ###

시스템 메시지는 어시스턴트에게 다양한 성격이나 행동을 부여하는 데 사용할 수 있습니다.

gpt-3.5-turbo-0301`은 일반적으로 `gpt-4-0314` 또는 `gpt-3.5-turbo-0613`만큼 시스템 메시지에 많은 주의를 기울이지 않는다는 점에 유의하세요. 따라서 `gpt-3.5-turbo-0301`의 경우 사용자 메시지에 중요한 지침을 대신 배치하는 것이 좋습니다. 일부 개발자는 대화가 길어질수록 모델의 주의가 분산되지 않도록 시스템 메시지를 대화 말미로 계속 이동시키는 데 성공했습니다.

In [7]:
# An example of a system message that primes the assistant to explain concepts in great depth
response = openai.ChatCompletion.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You are a friendly and helpful teaching assistant. You explain concepts in great depth using simple terms, and you give examples to help people learn. At the end of each explanation, you ask a question to check for understanding"},
        {"role": "user", "content": "Can you explain how fractions work?"},
    ],
    temperature=0,
)

print(response["choices"][0]["message"]["content"])


Sure! Fractions are a way of representing a part of a whole. The top number of a fraction is called the numerator, and it represents how many parts of the whole we are talking about. The bottom number is called the denominator, and it represents how many equal parts the whole is divided into.

For example, if we have a pizza that is divided into 8 equal slices, and we take 3 slices, we can represent that as the fraction 3/8. The numerator is 3 because we took 3 slices, and the denominator is 8 because the pizza was divided into 8 slices.

To add or subtract fractions, we need to have a common denominator. This means that the denominators of the fractions need to be the same. To do this, we can find the least common multiple (LCM) of the denominators and then convert each fraction to an equivalent fraction with the LCM as the denominator.

To multiply fractions, we simply multiply the numerators together and the denominators together. To divide fractions, we multiply the first fraction 

In [8]:
# An example of a system message that primes the assistant to give brief, to-the-point answers
response = openai.ChatCompletion.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You are a laconic assistant. You reply with brief, to-the-point answers with no elaboration."},
        {"role": "user", "content": "Can you explain how fractions work?"},
    ],
    temperature=0,
)

print(response["choices"][0]["message"]["content"])


Fractions represent a part of a whole. They consist of a numerator (top number) and a denominator (bottom number) separated by a line. The numerator represents how many parts of the whole are being considered, while the denominator represents the total number of equal parts that make up the whole.


### 몇 샷 프롬프트

어떤 경우에는 모델에게 원하는 것을 말하기보다 모델에게 원하는 것을 보여주는 것이 더 쉽습니다.

모델에게 원하는 것을 보여주는 한 가지 방법은 가짜 예제 메시지를 사용하는 것입니다.

예를 들어

In [9]:
# An example of a faked few-shot conversation to prime the model into translating business jargon to simpler speech
response = openai.ChatCompletion.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You are a helpful, pattern-following assistant."},
        {"role": "user", "content": "Help me translate the following corporate jargon into plain English."},
        {"role": "assistant", "content": "Sure, I'd be happy to!"},
        {"role": "user", "content": "New synergies will help drive top-line growth."},
        {"role": "assistant", "content": "Things working well together will increase revenue."},
        {"role": "user", "content": "Let's circle back when we have more bandwidth to touch base on opportunities for increased leverage."},
        {"role": "assistant", "content": "Let's talk later when we're less busy about how to do better."},
        {"role": "user", "content": "This late pivot means we don't have time to boil the ocean for the client deliverable."},
    ],
    temperature=0,
)

print(response["choices"][0]["message"]["content"])


We don't have enough time to complete the entire project perfectly.


예제 메시지가 실제 대화의 일부가 아니며 모델에서 다시 참조해서는 안 된다는 점을 명확히 하기 위해 '시스템' 메시지의 `이름` 필드를 `example_user` 및 `example_assistant`로 설정해 볼 수 있습니다.

위의 몇 가지 예시를 변형하면 다음과 같이 작성할 수 있습니다:

In [10]:
# The business jargon translation example, but with example names for the example messages
response = openai.ChatCompletion.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You are a helpful, pattern-following assistant that translates corporate jargon into plain English."},
        {"role": "system", "name":"example_user", "content": "New synergies will help drive top-line growth."},
        {"role": "system", "name": "example_assistant", "content": "Things working well together will increase revenue."},
        {"role": "system", "name":"example_user", "content": "Let's circle back when we have more bandwidth to touch base on opportunities for increased leverage."},
        {"role": "system", "name": "example_assistant", "content": "Let's talk later when we're less busy about how to do better."},
        {"role": "user", "content": "This late pivot means we don't have time to boil the ocean for the client deliverable."},
    ],
    temperature=0,
)

print(response["choices"][0]["message"]["content"])


This sudden change in plans means we don't have enough time to do everything for the client's project.


엔지니어링 대화를 시도하는 모든 시도가 처음부터 성공하는 것은 아닙니다.

첫 시도가 실패하더라도 모델을 준비하거나 컨디셔닝하는 다양한 방법을 실험해 보는 것을 두려워하지 마세요.

예를 들어, 한 개발자는 "지금까지 수고하셨습니다, 완벽했습니다"라는 사용자 메시지를 삽입하여 모델이 더 높은 품질의 응답을 제공하도록 컨디셔닝했을 때 정확도가 향상되는 것을 발견했습니다.

모델의 신뢰성을 높이는 방법에 대한 더 많은 아이디어가 필요하면 [신뢰성을 높이는 기술](../techniques_to_improve_reliability.md) 가이드를 읽어보세요. 이 가이드는 채팅이 아닌 모델을 위해 작성되었지만 많은 원칙이 여전히 적용됩니다.

## 4. 토큰 계산하기

요청을 제출하면 API가 메시지를 토큰 시퀀스로 변환합니다.

사용된 토큰의 개수는
- 요청 비용
- 응답을 생성하는 데 걸리는 시간
- 응답이 최대 토큰 한도에 도달하여 차단되는 경우(`gpt-3.5-turbo`의 경우 4,096개, `gpt-4`의 경우 8,192개)

다음 함수를 사용하여 메시지 목록에서 사용할 토큰 수를 계산할 수 있습니다.

메시지에서 토큰을 계산하는 정확한 방법은 모델마다 다를 수 있다는 점에 유의하세요. 아래 함수의 카운트는 영원한 보장이 아닌 추정치라고 생각하세요.

특히 입력한 선택적 함수를 사용하는 요청은 아래에서 계산된 예상치 외에 추가 토큰을 소비합니다.

토큰 카운트에 대한 자세한 내용은 [틱토큰으로 토큰 카운트하기](How_to_count_tokens_with_tiktoken.ipynb)에서 확인하세요.

In [11]:
import tiktoken


def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613"):
    """Return the number of tokens used by a list of messages."""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        print("Warning: model not found. Using cl100k_base encoding.")
        encoding = tiktoken.get_encoding("cl100k_base")
    if model in {
        "gpt-3.5-turbo-0613",
        "gpt-3.5-turbo-16k-0613",
        "gpt-4-0314",
        "gpt-4-32k-0314",
        "gpt-4-0613",
        "gpt-4-32k-0613",
        }:
        tokens_per_message = 3
        tokens_per_name = 1
    elif model == "gpt-3.5-turbo-0301":
        tokens_per_message = 4  # every message follows <|start|>{role/name}\n{content}<|end|>\n
        tokens_per_name = -1  # if there's a name, the role is omitted
    elif "gpt-3.5-turbo" in model:
        print("Warning: gpt-3.5-turbo may update over time. Returning num tokens assuming gpt-3.5-turbo-0613.")
        return num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613")
    elif "gpt-4" in model:
        print("Warning: gpt-4 may update over time. Returning num tokens assuming gpt-4-0613.")
        return num_tokens_from_messages(messages, model="gpt-4-0613")
    else:
        raise NotImplementedError(
            f"""num_tokens_from_messages() is not implemented for model {model}. See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens."""
        )
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3  # every reply is primed with <|start|>assistant<|message|>
    return num_tokens


In [13]:
# let's verify the function above matches the OpenAI API response

import openai

example_messages = [
    {
        "role": "system",
        "content": "You are a helpful, pattern-following assistant that translates corporate jargon into plain English.",
    },
    {
        "role": "system",
        "name": "example_user",
        "content": "New synergies will help drive top-line growth.",
    },
    {
        "role": "system",
        "name": "example_assistant",
        "content": "Things working well together will increase revenue.",
    },
    {
        "role": "system",
        "name": "example_user",
        "content": "Let's circle back when we have more bandwidth to touch base on opportunities for increased leverage.",
    },
    {
        "role": "system",
        "name": "example_assistant",
        "content": "Let's talk later when we're less busy about how to do better.",
    },
    {
        "role": "user",
        "content": "This late pivot means we don't have time to boil the ocean for the client deliverable.",
    },
]

for model in [
    "gpt-3.5-turbo-0301",
    "gpt-3.5-turbo-0613",
    "gpt-3.5-turbo",
    "gpt-4-0314",
    "gpt-4-0613",
    "gpt-4",
    ]:
    print(model)
    # example token count from the function defined above
    print(f"{num_tokens_from_messages(example_messages, model)} prompt tokens counted by num_tokens_from_messages().")
    # example token count from the OpenAI API
    response = openai.ChatCompletion.create(
        model=model,
        messages=example_messages,
        temperature=0,
        max_tokens=1,  # we're only counting input tokens here, so let's not waste tokens on the output
    )
    print(f'{response["usage"]["prompt_tokens"]} prompt tokens counted by the OpenAI API.')
    print()


gpt-3.5-turbo-0301
127 prompt tokens counted by num_tokens_from_messages().
127 prompt tokens counted by the OpenAI API.

gpt-3.5-turbo-0613
129 prompt tokens counted by num_tokens_from_messages().
129 prompt tokens counted by the OpenAI API.

gpt-3.5-turbo
129 prompt tokens counted by num_tokens_from_messages().
127 prompt tokens counted by the OpenAI API.

gpt-4-0314
129 prompt tokens counted by num_tokens_from_messages().
129 prompt tokens counted by the OpenAI API.

gpt-4-0613
129 prompt tokens counted by num_tokens_from_messages().
129 prompt tokens counted by the OpenAI API.

gpt-4
129 prompt tokens counted by num_tokens_from_messages().
129 prompt tokens counted by the OpenAI API.

